# X-Ray Landmark Detection on Google Colab

## Preperation

### Imports and installation of the required libraries

The libraries tensorboardx and bayesian-optimization are not within the virtual environment of Google Colab, hence they have to be installed manually.

In [1]:
from google.colab import drive
from google.colab import files
from zipfile import ZipFile
import os, glob

! pip install tensorboardx
! pip install bayesian-optimization


     |████████████████████████████████| 225kB 3.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/1d/0d/3b/6b9d4477a34b3905f246ff4e7acf6aafd4cc9b77d473629b77
Successfully built bayesian-optimization


### Google Colab or Zip upload
Either upload your project to Google Drive and mount it or upload project manually as .zip file and extract it.

In [2]:
use_google_drive = True

if use_google_drive:
  drive.mount('gdrive')
  % cd gdrive/My\ Drive/MLMI_SS19
else:
  file = files.upload()
  file_path = os.path.join(ROOT,list(file.keys())[0])
  zip_file = ZipFile(file_path)
  zip_file.extractall(ROOT)
  zip_file.close()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at gdrive
/content/gdrive/My Drive/MLMI_SS19


### Tensorboard and tunneling
Install ngrok for tunneling 

In [3]:
if os.path.exists("ngrok-stable-linux-amd64.zip"):
  os.remove("ngrok-stable-linux-amd64.zip")

if os.path.exists("ngrok"):
  os.remove("ngrok")
  
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-07-10 19:36:36--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.165.51.142, 52.207.111.186, 34.196.237.103, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.165.51.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17556757 (17M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  16.74M  26.8MB/s    in 0.6s    

2019-07-10 19:36:37 (26.8 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [17556757/17556757]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


Start tensorboard and forward port with ngrok

In [0]:
LOG_DIR = 'saved/log/'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6006 &')

Extract ngrok url for accessing tensorboard

**Attention**: Sometimes it throws an error like this:
```
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)
```
If this is the case the easiest way to solve this issue is to delete the ngrok*.zip and ngrok from the Google Drive folder and install them again.


In [5]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://b9960f3b.ngrok.io


## Training

### Imports

In [0]:
from config import CONFIG  
from parse_config import ConfigParser
from train import main

### Handle IOError

Google Colab has problems dealing with large amount of elements within a folder. Running it until it successfully loads will ensure there won't be an error later on. See [here](https://research.google.com/colaboratory/faq.html#drive-timeout) for further details.


In [0]:
data_folder = 'SubsetOnePatient' # 'OnePatient'

In [8]:
while True:
  try:
    os.listdir(f'data/XRay/{data_folder}/Training/ABD_LYMPH_005')
    os.listdir(f'data/XRay/{data_folder}/Validation/ABD_LYMPH_005')
  except IOError:
    print('IOError - keep running')
  else:
    print('succesfully accessed files')
    break;

succesfully accessed files


### Manual Training
Modify parameters and train model **manually**

In [0]:
CONFIG['arch']['args']['x_channels'] = 128
CONFIG['arch']['args']['stage_channels'] = 512
CONFIG['arch']['args']['num_stages'] = 5
CONFIG['arch']['args']['dilation'] = 1
CONFIG['arch']['args']['depthwise_separable_convolution'] = True

CONFIG['data_loader']['args']['data_dir'] = f'data/XRay/{data_folder}'
CONFIG['data_loader']['args']['batch_size'] = 1
CONFIG['data_loader']['args']['validation_split'] = 0.2
CONFIG['data_loader']['args']['shuffle'] = False
CONFIG['data_loader']['args']['custom_args']['fraction_of_dataset'] = 1
CONFIG['data_loader']['args']['custom_args']['sigma'] = 80
CONFIG['data_loader']['args']['custom_args']['sigma_reduction_factor'] = 0.9

CONFIG['optimizer']['args']['lr'] = 1e-5

CONFIG['trainer']['epochs'] = 1000
CONFIG['trainer']['save_period'] = 1
CONFIG['trainer']['early_stop'] = 50

CONFIG['prediction_blur'] = 2

main(ConfigParser(CONFIG))

### Resume training
By default it takes your last training run and the last model of it. 
If you want to use a specific run or a specific model you can provide it like this:

```
run_dir = "0629_194146"
model_pth = "checkpoint-epoch11.pth"
```



In [0]:
from importlib.machinery import SourceFileLoader

base_saved_dir = "saved/models/XRay"

run_dir = None
model_pth = None

for temp_run_dir in os.listdir(base_saved_dir)[::-1]:
  if run_dir is None:
      run_path = os.path.join(base_saved_dir, temp_run_dir)
  else:
    run_path = os.path.join(base_saved_dir, run_dir)

  if model_pth is None:
    model_path_list = glob.glob(f'{run_path}/checkpoint-epoch*.pth')
    if not model_path_list:
      continue
    model_path = model_path_list[-1]
    break
  else:
    model_path = os.path.join(run_path, model_pth)
    break

config = SourceFileLoader("CONFIG", os.path.join(run_path, 'config.py')).load_module().CONFIG
epoch = int(model_path.split('checkpoint-epoch')[-1][:-4])
config['data_loader']['args']['custom_args']['sigma'] *= config['data_loader']['args']['custom_args']['sigma_reduction_factor'] ** epoch
main(ConfigParser(config, model_path))


ConvolutionalPoseMachines(
  (stage_1): Stage1(
    (X): X(
      (convs): ModuleList(
        (0): Conv2d(1, 128, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
        (1): DepthwiseSeparableConvolution(
          (depthwise): Conv2d(128, 128, kernel_size=(9, 9), stride=(1, 1))
          (pointwise): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), padding=(4, 4))
          (relu): ReLU()
        )
        (2): DepthwiseSeparableConvolution(
          (depthwise): Conv2d(128, 128, kernel_size=(9, 9), stride=(1, 1))
          (pointwise): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), padding=(4, 4))
          (relu): ReLU()
        )
        (3): DepthwiseSeparableConvolution(
          (depthwise): Conv2d(128, 128, kernel_size=(5, 5), stride=(1, 1))
          (pointwise): Conv2d(128, 32, kernel_size=(1, 1), stride=(1, 1), padding=(2, 2))
          (relu): ReLU()
        )
      )
      (max_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)

### Bayesian Optimization
Do **automatic** Bayesian optimization

In [0]:
from bayes_opt_train import run_bayes_opt

run_bayes_opt({
    'num_channels': (6, 8),  # {64, 128, 256}
    'num_stacks': (2, 7),
    'num_blocks': (1, 7),
    'kernel_size': (1, 4),  # {3, 5, 7, 9}
    'sigma': (0.6, 5),
    'prediction_blur': (0.01, 1),
    'threshold': (0.00001, 0.2),
    'epochs': (200, 200)

}, init_points=10, n_iter=10)